Notebook to explore a way to analyse site text to identify activities of workers

In [1]:
import json
import os
from pandas.io.json import json_normalize
import pandas as pd

# function to convert json file to dataframe
def json2df(file):
    tweets = []
    for line in open(file, encoding="utf8"):
        tweets.append(json.loads(line))

    df = pd.DataFrame(tweets)
    return df

alljsonsites_filename = 'scraping_websites_parse_workers_webites_results.jl'
dirpath = os.getcwd()
data_foldername = dirpath+ "\data"
alljsonsites_filename_path = data_foldername+"\\"+alljsonsites_filename 

df = json2df(alljsonsites_filename_path)

print(df.tail(2))


  base_scheme                     base_url                               dom  \
3      [http]          [www.calorbois.com]  [R08-poêle, R08-Entretien poêle]   
4      [http]  [www.samson-couverture.com]                    [R08-cheminée]   

                                          image_urls  \
3  [http://www.calorbois.com/img/logo/logo.jpg, h...   
4  [http://www.samson-couverture.com/images/logoc...   

                                              images             lib_arty  \
3  [{'url': 'http://www.calorbois.com/img/logo/lo...                  NaN   
4  [{'url': 'http://www.samson-couverture.com/ima...  [SAMSON COUVERTURE]   

                                siret  \
3                                 NaN   
4  [http://www.samson-couverture.com]   

                                                 url  \
3  [http://www.calorbois.com/mentions%20legales.h...   
4                 [http://www.samson-couverture.com]   

                                 website_description           we

In [38]:
#On ajoute une colonne avec les colonnes de texte qui nous intéresse
import re
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
        self.convert_charrefs = True
        self.entityref = re.compile('&[a-zA-Z][-.a-zA-Z0-9]*[^a-zA-Z0-9]')

    def handle_data(self, d):
        self.fed.append(d)

    def handle_starttag(self, tag, attrs):
        #remplace les tags de début par un espace
        self.fed.append(' ')

    def handle_endtag(self, tag):
        #remplace les tags de fin par un espace
        self.fed.append(' ')

    def handle_entityref(self, name):
        if html.parser.get(name) is None:
            m = self.html.parser.match(self.rawdata.splitlines()[self.lineno-1][self.offset:])
            entity = m.group()
            # semicolon is consumed, other chars are not.
            if entity[-1] != ';':
                entity = entity[:-1]
            self.fed.append(entity)
        else:
            self.fed.append(' ')

    def get_data(self):
        self.close()    # N.B. ensure all buffered data has been processed
        return ''.join(self.fed)

def strip_tags(html):
    s = MyHTMLParser()
    s.feed(html)
    data = s.get_data()
    s.close()
    return data


print(strip_tags('<title>html &zzz; <div> &eacute; tags<p>&zzz &zz: will be&amp;replaced</p>with space. NOT this &abc'))


 html &zzz;   é tags &zzz &zz: will be&replaced with space. NOT this &abc


In [39]:
# convertit dans le dataframe 
df['website_titles']=df['website_titles'].apply(lambda x: strip_tags(str(x)))

print(df['website_titles'].head(10))


0         [' calor bois - philippe loquin - accueil ']
1                            [' froid systemservice ']
2    [' calor bois - philippe loquin - contact et c...
3                  [' calor bois - mentions légales ']
4    [' samson couverture zinguerie à angers, maine...
Name: website_titles, dtype: object


In [40]:
# Start text classification
import spacy
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

import string
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.fr import French

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('fr_core_news_sm')
stop_words = spacy.lang.fr.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = French()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

unephrase=str(df['website_description'].iloc[4])
print('description : ',unephrase)
print('token de cette description : ', spacy_tokenizer(unephrase))

description :  ['nombreux services pour tout ce qui concerne la toiture, en création, rénovation et entretien']
token de cette description :  ['service', 'concerner', 'toiture', 'création', 'rénovation', 'entretien']


In [41]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    text = ''.join(text)

    # Removing spaces and converting text into lowercase
    return text.strip().lower()


# Vectorization Feature Engineering (TF-IDF)
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [42]:
#Splitting The Data into Training and Test Sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

X = df['website_description']

ylabels = df['dom']
ylabels = MultiLabelBinarizer().fit_transform(ylabels)
print(ylabels)
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

[[1 1 1]
 [1 1 1]
 [1 0 1]
 [1 0 1]
 [0 1 0]]


In [43]:
#Creating a Pipeline and Generating the Model
# Run classifier
from sklearn import svm, datasets
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

random_state = np.random.RandomState(0)

classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
                                 random_state=random_state))
 
# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)


Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x00000000186EB588>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=No...
                                 vocabulary=None)),
                ('classifier',
                 OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200,
                                                   class_weight=None, coef0=0.0,
                                                   decision_function_shape='ovr',
                                                   degree=3,
   